In [ ]:
import tensorflow as tf
import tensorflow_datasets as tfds
from keras.layers import Layer, Input
from keras.layers import Conv2D, Dense, MaxPooling2D, BatchNormalization, Flatten, Dropout, Conv2DTranspose
from keras.preprocessing.image import img_to_array, load_img
from keras.models import Model, Sequential
from keras.layers import Concatenate, UpSampling2D
import cv2
import os
import random
import numpy as np
import matplotlib.pyplot as plt

# Load Images


In [ ]:
path = "/kaggle/input/semantic-segmentation-of-aerial-imagery/Semantic segmentation dataset"
listDir = []
for dir in os.listdir(path):
    if dir != "classes.json":
        listDir.append(dir)

listDir = sorted(listDir)

In [ ]:
def loadImg(path, dirTile):
    input_img, mask_img = [], []
    path = os.path.join(path, dirTile)
    inpDir, maskDir = os.listdir(path)
    
    imgDirs = sorted(os.listdir(os.path.join(path, inpDir)))
    mask_imgDirs = sorted(os.listdir(os.path.join(path, maskDir)))
    
    for imgDir in imgDirs:
        imgDir = os.path.join(path, inpDir, imgDir)
        img = load_img(imgDir, target_size = (224, 224))
        img = tf.convert_to_tensor(img, dtype = tf.float32) / 255.0
        input_img.append(img)
    
    for mask_imgDir in mask_imgDirs:
        mask_imgDir = os.path.join(path, maskDir, mask_imgDir)
        mask = load_img(mask_imgDir, target_size = (224, 224))
        mask = tf.convert_to_tensor(mask, dtype = tf.float32) / 255.0
        mask_img.append(mask)
    
    return input_img, mask_img

In [ ]:
def loadData(path = path, listDir = listDir):
    imgs_data, masks_data = [], []
    for dirTile in listDir:
        input_imgs, mask_imgs = loadImg(path, dirTile)
        
        imgs_data.extend(input_imgs)
        masks_data.extend(mask_imgs)
    return imgs_data, masks_data

In [ ]:
input_imgs, mask_imgs = loadData()

In [ ]:
plt.figure(figsize = (10, 6))
plt.subplot(1, 2, 1)
plt.title("Input")
plt.imshow(input_imgs[0])
plt.axis("off")

plt.subplot(1, 2, 2)
plt.title("Mask")
plt.imshow(mask_imgs[0])
plt.axis("off")

In [ ]:
from sklearn.model_selection import train_test_split

train_imgs, test_imgs, train_masks, test_masks = train_test_split(input_imgs, mask_imgs, 
                                                                  test_size = 0.2, 
                                                                  shuffle = True, 
                                                                  random_state = 42)

In [ ]:
plt.subplot(1, 2, 1)
plt.imshow(train_imgs[1])

plt.subplot(1, 2, 2)
plt.imshow(train_masks[1])

In [ ]:
train_data = np.array(list(zip(train_imgs, train_masks)))
test_data = np.array(list(zip(test_imgs, test_masks)))

# Model (from Scratch)


In [ ]:
def Conv(x, dropout, filters):
  x = Conv2D(filters = filters, kernel_size = 3, padding = 'same', activation = 'relu')(x)
  x = BatchNormalization()(x)
  x = Conv2D(filters = filters, kernel_size = 3, padding = 'same', activation = 'relu')(x)
  x = Dropout(dropout)(x)
  pool = MaxPooling2D(pool_size = (2, 2))(x)

  return x, pool

def UpSample(x, skip_connect, dropout, filters):
  x = Concatenate(axis = -1)([skip_connect, UpSampling2D(size = (2, 2))(x)])
  x = Conv2D(filters = filters, kernel_size = 3, padding = 'same', activation = 'relu')(x)
  x = Dropout(dropout)(x)
  x = BatchNormalization()(x)
  x = Conv2D(filters = filters, kernel_size = 3, padding = 'same', activation = 'relu')(x)

  return x


In [ ]:
def UNet(x, dropout, conv = [32, 64, 128, 256, 512]):

  # encoder
  conv1, pool1 = Conv(x, dropout, conv[0]) # 224 -> 112
  conv2, pool2 = Conv(pool1, dropout, conv[1]) # 112 -> 56
  conv3, pool3 = Conv(pool2, dropout, conv[2]) # 56 -> 28
  conv4, pool4 = Conv(pool3, dropout, conv[3]) # 28 -> 14

  conv5 = Conv2D(filters = conv[4], kernel_size = 3, padding = 'same', activation = 'relu')(pool4)
  conv5 = Dropout(dropout)(conv5)
  conv5 = BatchNormalization()(conv5)

  # decoder
  conv6 = UpSample(conv5, conv4, dropout, conv[4]) # 14 -> 28
  conv7 = UpSample(conv6, conv3, dropout, conv[3]) # 28 -> 56
  conv8 = UpSample(conv7, conv2, dropout, conv[2]) # 56 -> 112
  conv9 = UpSample(conv8, conv1, dropout, conv[1]) # 112 -> 224

  # output
  result = Conv2D(filters = 3, kernel_size = 1, padding = 'same', activation = 'sigmoid')(conv9)

  return result

In [ ]:
input = Input(shape = (224, 224, 3))
output = UNet(input, dropout = 0.1)

model = Model(inputs = input, outputs = output)
model.summary()

In [ ]:
from keras.callbacks import EarlyStopping

earlystopping = EarlyStopping(monitor = 'val_loss',
                              min_delta = 0,
                              patience = 2,
                              verbose = 0,
                              mode = 'auto')

In [ ]:
model.compile(optimizer = "Adam", loss = "binary_crossentropy", metrics = ["accuracy"])

In [ ]:
history = model.fit(np.array(train_imgs), np.array(train_masks),
                    batch_size = 8, 
                    epochs = 100, 
                    validation_data = (np.array(test_imgs), np.array(test_masks)),
                    callbacks = [earlystopping])

# Result Visualization

In [ ]:
plt.style.use('seaborn')
plt.figure(figsize = (11, 8))
plt.plot(history.history['loss'], c = 'orange', label = 'train_loss')
plt.plot(history.history['val_loss'], c = 'blue', label = 'val_loss')
plt.legend()
plt.show()

In [ ]:
plt.style.use('seaborn')
plt.figure(figsize = (11, 8))
plt.plot(history.history['accuracy'], c = 'orange', label = 'train_acc')
plt.plot(history.history['val_accuracy'], c = 'blue', label = 'val_acc')
plt.legend()
plt.show()

# Predict

In [ ]:
images_predict = model.predict(np.array(test_imgs))

In [ ]:
def show_result(id):
    fig, ax = plt.subplots(1, 3, figsize = (12, 7))
    
    ax[0].set_title('original')
    ax[0].imshow(test_imgs[id])
    ax[0].axis("off")
    
    ax[1].set_title('predict')
    ax[1].imshow(images_predict[id])
    ax[1].axis("off")
    
    ax[2].set_title('ground_truth')
    ax[2].imshow(test_masks[id])
    ax[2].axis("off")

In [ ]:
for id in range(5):
    show_result(id)

# Model (Pre-trained)

In [ ]:
from keras.applications.mobilenet_v2 import MobileNetV2

model = MobileNetV2(input_shape = (224, 224, 3),
                    include_top = False,
                    weights = 'imagenet')

In [ ]:
model.summary()

## UNet implement

In [ ]:
import keras
def down_path():
    layers = ['block_1_expand_BN',
              'block_2_expand_BN',
              'block_4_expand_BN',
              'block_12_expand_relu',
              'block_16_depthwise_relu']
    outputs = [model.get_layer(layer).output for layer in layers]
    encoder = Model(inputs = model.input, outputs = outputs)
    encoder.trainable = False
    
    return encoder

def up_path():
    decoder = []
    for filters in [512, 256, 128, 64]:
        block = Sequential([
            Conv2DTranspose(filters = filters, kernel_size = 3, strides = 2, padding = 'same'),
            BatchNormalization(),
            keras.layers.ReLU(),
            Dropout(0.1)
        ])
        decoder.append(block)
    return decoder

def UNetModel(input = Input(shape = (224, 224, 3))):
    encoder = down_path()
    decoder = up_path()
    
    outputs = encoder(input)
    
    output = outputs[-1]
    encode_outputs = reversed(outputs[:-1])
    
    for id, encode_output in enumerate(encode_outputs):
        decode_output = decoder[id](output)
        output = Concatenate()([encode_output, decode_output])
        
    output = Conv2DTranspose(filters = 32, kernel_size = 3, strides = 2, padding = "same")(output)
    output = BatchNormalization()(output)
    output = Conv2D(filters = 3, kernel_size = 3, strides = 1, padding = 'same', activation = 'sigmoid')(output)
    
    model = Model(inputs = input, outputs = output)
    return model

In [ ]:
model = UNetModel()
model.summary()

In [ ]:
model.compile(optimizer = 'adam', loss = keras.losses.binary_crossentropy, metrics = ['accuracy'])

In [ ]:
h = model.fit(np.array(train_imgs), np.array(train_masks),
              batch_size = 8,
              epochs = 15,
              validation_data = (np.array(test_imgs), np.array(test_masks)))
#               callbacks = [earlystopping])

# Result visualization

In [ ]:
plt.style.use('seaborn')
plt.figure(figsize = (11, 8))
plt.plot(h.history['loss'], c = 'orange', label = 'train_loss')
plt.plot(h.history['val_loss'], c = 'blue', label = 'val_loss')
plt.legend()
plt.show()

In [ ]:
plt.style.use('seaborn')
plt.figure(figsize = (11, 8))
plt.plot(history.history['accuracy'], c = 'orange', label = 'train_acc')
plt.plot(history.history['val_accuracy'], c = 'blue', label = 'val_acc')
plt.legend()
plt.show()

# Predict

In [ ]:
images_predict = model.predict(np.array(test_imgs))

In [ ]:
def show_result(id):
    fig, ax = plt.subplots(1, 3, figsize = (12, 12))
    ax[0].set_title("Origin")
    ax[0].imshow(test_imgs[id])
    ax[0].axis("off")
    
    ax[1].set_title("Predict")
    ax[1].imshow(images_predict[id])
    ax[1].axis("off")
    
    ax[2].set_title("Ground truth")
    ax[2].imshow(test_masks[id])
    ax[2].axis("off")

In [ ]:
for id in range(5):
    show_result(id)